This colab notebook is based on https://huggingface.co/docs/smolagents/examples/text_to_sql


In [1]:
!pip install -q "smolagents[transformers]" "transformers>=4.43" "accelerate>=0.33" bitsandbytes pandas sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 16.1 MB/s eta 0:00:00


In [2]:
import bitsandbytes as bnb
print("bitsandbytes version:", bnb.__version__)

bitsandbytes version: 0.47.0


1) SQLite

In [3]:
# Create an in-memory SQLite database
# define a receipts table in memory, insert some rows, and then you can interact with it using SQLAlchemy

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)

metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

In [4]:
#To verify it worked, you can query the table back

from sqlalchemy import select

# Build a query
stmt = select(receipts)
with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()

for row in results:
    print(row)


(1, 'Alan Payne', 12.06, 1.2)
(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)


In [5]:
from sqlalchemy import func

stmt = select(func.avg(receipts.c.price), func.avg(receipts.c.tip))
with engine.connect() as connection:
    avg_price, avg_tip = connection.execute(stmt).one()
print("Avg price:", avg_price, " | Avg tip:", avg_tip)


Avg price: 27.615  | Avg tip: 1.9674999999999998


In [6]:
inspector = inspect(engine)
print(inspector.get_columns("receipts"))


[{'name': 'receipt_id', 'type': INTEGER(), 'nullable': False, 'default': None, 'primary_key': 1}, {'name': 'customer_name', 'type': VARCHAR(length=16), 'nullable': False, 'default': None, 'primary_key': 2}, {'name': 'price', 'type': FLOAT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'tip', 'type': FLOAT(), 'nullable': True, 'default': None, 'primary_key': 0}]


In [7]:
stmt = select(receipts).where(receipts.c.price > 20)
with engine.connect() as connection:
    for row in connection.execute(stmt):
        print(row)


(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)


In [8]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


2. Create a SQL tool for the agent

In [9]:
# create a SQLite tool for the agent

from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

3. LLM

In [10]:
# use an open-source LLM (local, via TransformersModel)

import torch

from smolagents import CodeAgent, TransformersModel

MODEL_ID = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
#MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"

model = TransformersModel(
    model_id=MODEL_ID,
    device_map="auto",
    torch_dtype="auto" if torch.cuda.is_available() else None,
    max_new_tokens=512,
    temperature=0.2,
    trust_remote_code=True,
    model_kwargs={"load_in_4bit": True}
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

4. Agent

In [11]:
agent = CodeAgent(
    tools=[sql_engine],
    model=model)


print("Agent ready with model:", MODEL_ID)
print(sql_engine.description)

Agent ready with model: HuggingFaceTB/SmolLM2-1.7B-Instruct
Allows you to perform SQL queries on the table. Returns a string representation of the result.
The table is named 'receipts'. Its description is as follows:
    Columns:
    - receipt_id: INTEGER
    - customer_name: VARCHAR(16)
    - price: FLOAT
    - tip: FLOAT


5. Queries

In [12]:
agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ TransformersModel - HuggingFaceTB/SmolLM2-1.7B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  answer = sql_engine(query="SELECT customer_name FROM receipts WHERE price = (SELECT MAX(price) FROM receipts)")  
  final_answer(answer)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 
('Woodrow Wilson',)

[Step 1: Duration 4.85 seconds| Input tokens: 2,326 | Output tokens: 69]

"\n('Woodrow Wilson',)"